In [12]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [13]:
# parameters
p = 18 # nr of random variables
n = 1000 # nr of realizations
mean = 0
sigma = 1.1

Covariance matrix is given by $ \textbf{C} = \hat{\lambda}^* \textbf{T} + (1-\hat{\lambda}^*)\textbf{S} $, where $\hat{\lambda}^*$ is the estimated optimal shrinkage intensity, $\textbf{T}$ is the target matrix which we take to be the identity, and $\textbf{S}$ is the empirically estimated covariance matrix.

We run a 100 simulations to find the shrinkage intensity.

In [18]:
T = np.identity(p) # Target matrix
C_true = sigma**2 * np.identity(p) # True covariance

In [23]:
lmbdas_est = []
MSE_MC_sum = 0
MSE_MCS_sum = 0

m = 100 # nr of realizations
for _ in range(m):
    X = np.random.normal(mean, sigma, size=(p, n))
    
    x_mean = np.sum(X, axis=1)/n # Find mean of each row
    x_mean_M = np.tile(x_mean, (n, 1)).T # Repeat mean values as columns in a p x n matrix
    Y = X - x_mean_M
    
    W = []
    # Generate W array (which is 3D) of size (n, p, p), order of indices (k, i, j)
    for k in range(n):
        w = Y[:,k]
        W.append(np.outer(w, w))
    W_mean = np.sum(W, axis=0)/n
    
    # Emperically estimated covariance matrix
    S = n / (n-1) * W_mean
    
    W_mean_rep = np.tile(W_mean, (n, 1, 1))
    V = W - W_mean_rep
    # Compute variance of elements of the covariance matrix
    Var = n / (n-1)**3 * np.sum(V**2, axis=0)
    
    lmbda_est = np.sum(Var) / np.sum((T-S)**2)
    lmbdas_est.append(lmbda_est)
    
    # Compute shrinkage covariance matrix
    C_shrinkage = lmbda_est*T + (1-lmbda_est)*S
    
    MSE_MC = np.trace((S-C_true)@(S-C_true).T)
    MSE_MCS = np.trace((C_shrinkage-C_true)@(C_shrinkage-C_true).T)
    MSE_MC_sum += MSE_MC
    MSE_MCS_sum += MSE_MCS

MSE_MC = MSE_MC_sum / m
MSE_MCS = MSE_MCS_sum / m

lmbda_est = np.average(lmbdas_est)
print(f"Optimal value for shrinkage intensity: {lmbda_est}")
lmbda_est_se = np.std(lmbdas_est) #/ np.sqrt(len(lmbdas_est))
print(f"Standard deviation: {lmbda_est_se}")

print(f"MSE for MC: {MSE_MC}")
print(f"MSE for MC+S: {MSE_MCS}")

Optimal value for shrinkage intensity: 0.3865124672942543
Standard deviation: 0.02596147517275789
MSE for MC: 0.5048506253068621
MSE for MC+S: 0.3094746950131999
